In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from pycocotools.coco import COCO
from sklearn.metrics import precision_score, recall_score
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models

from Dataset import MyCOCO
from Train import train_model

run_transfer = False

In [3]:
# Load the datasets

datasets = {x: MyCOCO(mode = x) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size = 32, shuffle = True, num_workers = 4) for x in ['train', 'val']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}

loading annotations into memory...
Done (t=9.66s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!


In [4]:
model_choice = 1

if model_choice == 0:
    model = models.resnet18(pretrained = True)
elif model_choice == 1:
    model = models.mobilenet_v2(pretrained = True)

for param in model.parameters():
    param.requires_grad = False

if model_choice == 0:
    model.fc = torch.nn.Linear(in_features = 512, out_features = 91)
    optim_params = model.fc.parameters()
elif model_choice == 1:
    model.classifier[1] = torch.nn.Linear(in_features = 1280, out_features = 91)
    optim_params = model.classifier.parameters()
    
print(model)


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [5]:

model.cuda()

if run_transfer:

    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer = optim.Adam(optim_params, lr= 0.001)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 2, gamma = 0.1)

    model = train_model(model, dataloaders, dataset_sizes, criterion, optimizer, exp_lr_scheduler, num_epochs = 5)

    torch.save(model.state_dict(), './model_transfer.pt')
    
else:
    model.load_state_dict(torch.load('./model_transfer.pt'))
    
model.eval()


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [6]:
# Setup COCO
coco = COCO('/home/gregory/Datasets/COCO/annotations/instances_val2017.json')
cats = coco.loadCats(coco.getCatIds())


loading annotations into memory...
Done (t=0.32s)
creating index...
index created!


In [7]:
# Get the validation predictions and labels

y_hat = []
y_true = []

for inputs, labels in dataloaders['val']:
    
    y_hat.append(1.0 * (model(inputs.cuda()).cpu().data.numpy() > 0.0))
        
    y_true.append(labels.numpy())
        
y_hat = np.concatenate(np.array(y_hat), axis = 0)
y_true = np.concatenate(np.array(y_true), axis = 0)

dim = y_hat.shape[1]

In [8]:
# Calculate Precision and Recall

precision = np.zeros((dim))

for i in range(dim):
    precision[i] = precision_score(np.squeeze(y_true[:, i]), np.squeeze(y_hat[:, i]), zero_division = 0)
    

recall = np.zeros((dim))

for i in range(dim):
    recall[i] = recall_score(np.squeeze(y_true[:, i]), np.squeeze(y_hat[:, i]), zero_division = 0)
    

print("Object Precision Recall")
MAP = 0.0
MAR = 0.0
for cat in cats:
    p = precision[cat['id']]
    MAP += p
    r = recall[cat['id']]
    MAR += r
    print(cat['name'], p, r)
    
    
print("MAP MAR")
print(MAP / len(cats), MAR / len(cats))


Object Precision Recall
person 0.9013798701298701 0.824730783512811
bicycle 0.717948717948718 0.18791946308724833
car 0.6959706959706959 0.35514018691588783
motorcycle 0.8080808080808081 0.5031446540880503
airplane 0.8108108108108109 0.6185567010309279
bus 0.7946428571428571 0.4708994708994709
train 0.8785046728971962 0.5987261146496815
truck 0.6276595744680851 0.236
boat 0.7313432835820896 0.4049586776859504
traffic light 0.6947368421052632 0.34554973821989526
fire hydrant 0.896551724137931 0.3023255813953488
stop sign 0.8 0.4057971014492754
parking meter 0.6666666666666666 0.21621621621621623
bench 0.6571428571428571 0.09787234042553192
bird 0.9655172413793104 0.224
cat 0.8650793650793651 0.592391304347826
dog 0.8181818181818182 0.3050847457627119
horse 0.8 0.40625
sheep 0.8636363636363636 0.5846153846153846
cow 0.875 0.40229885057471265
elephant 0.8902439024390244 0.8202247191011236
bear 0.9142857142857143 0.6530612244897959
zebra 0.9156626506024096 0.8941176470588236
giraffe 0.9062